# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [13]:
# your code here
sub1 = orders.groupby('CustomerID').agg('sum')
sub1 = sub1['amount_spent'].sort_values(ascending=False).reset_index()
sub1[:3]

,CustomerID,amount_spent
0,14646,280206.02
1,18102,259657.30
2,17450,194550.79


In [12]:
q25=sub1['amount_spent'].quantile(0.25)
q75=sub1['amount_spent'].quantile(0.75)

IQR=q75-q25
sub2=sub1[(sub1['amount_spent']>=q25)&(sub1['amount_spent']<q75)]
sub2[:3]

,CustomerID,amount_spent
1085,15214,1661.44
1086,15024,1661.33
1087,15332,1661.06


In [21]:
q95=sub1['amount_spent'].quantile(0.95)
q80=sub1['amount_spent'].quantile(0.8)
preferred=sub1[(sub1['amount_spent']>=q80)&(sub1['amount_spent']<q95)]
vip=sub1[(sub1['amount_spent']>=q95)]

list_p=list(preferred['CustomerID'])
list_v=list(vip['CustomerID'])
col=[]
for e in sub1.CustomerID:
    if e in list_p:
        col.append('Preferencial')
    elif e in list_v:
        col.append('VIP')
    else:
        col.append('Regular')

sub1['Clase']=col
sub1.sample(7)

,CustomerID,amount_spent,Clase
2407,15550,575.63,Regular
853,17490,2092.32,Preferencial
99,14062,10464.85,VIP
2223,14548,652.80,Regular
807,15797,2207.40,Preferencial
3790,17253,178.26,Regular
963,14887,1862.00,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [22]:
# your code here
country_v=sub1.merge(orders[['CustomerID','Country']],how='left',on='CustomerID').drop_duplicates()
country_v[:3]

,CustomerID,amount_spent,Clase,Country
0,14646,280206.02,VIP,Netherlands
2080,18102,259657.30,VIP,United Kingdom
2511,17450,194550.79,VIP,United Kingdom


In [32]:
#Q3
vips=country_v[country_v['Clase']=='VIP'].groupby('Country').count()
vips=vips['CustomerID'].sort_values(ascending=False).reset_index()
prefs=country_v[country_v['Clase']=='VIP'].groupby('Country').count()
prefs=prefs['CustomerID'].sort_values(ascending=False).reset_index()

vps=vips.reset_index().merge(prefs, how='left',on='Country')
vps['Total']=vps.CustomerID_x+vps.CustomerID_y
vps[:1]

,index,Country,CustomerID_x,CustomerID_y,Total
0,0,United Kingdom,177,177,354
